In [21]:
import re

# try a different parser
import srsly
import json
import spacy
nlp = spacy.load("en_core_web_md")
# nlp.get_pipe("parser")

with open("../../data/resume.jsonl", "r", encoding="utf-8") as f1:
    resume = json.load(f1)

    sent_parse = []
    for k in resume:
        doc = nlp(k["text"])
        for s in doc.sents:
            if len(s) > 2:
                try:
                    # f = re.sub(r"\s\-\s+", "", s)
                    sent_parse.append({'text': f"{s}"})
                except Exception as e:
                    continue

srsly.write_json("../../data/resume_sentences_parser.jsonl", sent_parse)

In [17]:
import spacy
import random
# create a silver annotated dataset with sentences (train/test)

'''
Rehearsal updates run an initial copy of the model over some data
'''
nlp = spacy.load("en_core_web_md")

with open("../../data/resume_sentences_parser.jsonl", "r", encoding="utf-8") as f1:
    resume = json.load(f1)

    limit = 0.2
    train = list()
    test = list()
    for k in resume:
        doc = nlp(k["text"])
        for sent in doc.sents:
            labels = list()
            for e in sent.ents:
                labels.append([e.start_char, e.end_char, e.label_])
            if labels:
                spacy_entry = (sent.text, {"entities": labels})
                if random.uniform(0, 1) > limit:
                    train.append(spacy_entry)
                else:
                    test.append(spacy_entry)

    srsly.write_json("../../data/annotated/rehearse_silver_sent_train.jsonl", train)
    srsly.write_json("../../data/annotated/rehearse_silver_sent_test.jsonl", test)